In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.9 MB/s eta 0:00

In [2]:
import torch
import datasets
import numpy as np

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("xsum")


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
test_data = raw_datasets['test']

In [5]:
test_data

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11334
})

In [6]:
train_data = raw_datasets['train']

In [7]:
train_data[0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [8]:
from datasets import DatasetDict
raw_datasets = DatasetDict(
    {
        "train": train_data.select(range(300)),
       
    }
)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 300
    })
})

In [9]:
def remove_newline(data):
  data["document"] = data["document"].replace('\n', '')
  data["summary"] = data["summary"].replace('\n', '')
  return data

In [10]:
raw_datasets = raw_datasets.map(remove_newline)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [11]:
next(iter(raw_datasets["train"]))

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.First Minister Nicola Sturgeon visited the area to inspect the damage.The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.However, she said more preventative work could have been carried out to ensure the retaining wall did not fail."It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it

In [12]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import torch.nn as nn
from transformers.optimization import Adafactor
#from soft_prompt_tuning.soft_embedding import SoftEmbedding
import transformers
import torch
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2').to("cuda")

In [13]:
tokenizer.add_special_tokens({'pad_token':'[PAD]'})

1

In [120]:
prompt_tokens = tokenizer(raw_datasets["train"]['document'], return_tensors="pt",padding=True)
prompt_len = prompt_tokens.input_ids.shape[1]
target_tokens = tokenizer(raw_datasets["train"]['summary'], return_tensors="pt",padding=True)
target_len = target_tokens.input_ids.shape[1]

In [22]:
print(prompt_tokens['input_ids'].shape)
print(prompt_len)

torch.Size([300, 2667])


In [23]:
prefix_len = 100
target_start = prefix_len + prompt_len

In [26]:
inputs_token = tokenizer(raw_datasets["train"]['document'])


In [28]:
len(inputs_token['input_ids'])

300

In [33]:
target_tokens['input_ids'].shape

torch.Size([300, 53])

In [34]:
inputs_token['input_ids'] = torch.cat([torch.full((300,prefix_len), 5555), prompt_tokens['input_ids'], target_tokens['input_ids']],1).cuda()

In [42]:
inputs_token['input_ids'].shape

torch.Size([300, 2820])

In [40]:
inputs_token['attention_mask'] = torch.cat([torch.full((300,prefix_len), 1), prompt_tokens['attention_mask'], target_tokens['attention_mask']],1).cuda()
inputs_token['attention_mask'].shape

torch.Size([300, 2820])

In [41]:
labels = torch.cat([torch.full((300,target_start), -100), target_tokens['input_ids']], 1).cuda()
labels.shape

torch.Size([300, 2820])

In [52]:
labels

tensor([[ -100,  -100,  -100,  ..., 50257, 50257, 50257],
        [ -100,  -100,  -100,  ..., 50257, 50257, 50257],
        [ -100,  -100,  -100,  ..., 50257, 50257, 50257],
        ...,
        [ -100,  -100,  -100,  ..., 50257, 50257, 50257],
        [ -100,  -100,  -100,  ..., 50257, 50257, 50257],
        [ -100,  -100,  -100,  ..., 50257, 50257, 50257]], device='cuda:0')

In [108]:
def collate_batch(batch):
    src_batch, attention_batch,  tgt_batch = [], [], [] 
    for src_sample, attn_sample, trg_sample in batch:
        src_batch.append(src_sample)
        attention_batch.append(attn_sample)
        tgt_batch.append(trg_sample)
    
    src_batch = torch.stack(src_batch)
    attention_batch = torch.stack(attention_batch)
    tgt_batch = torch.stack(tgt_batch)
    return src_batch,attention_batch, tgt_batch

In [109]:
from torch.utils.data import DataLoader

In [110]:
loader = DataLoader(list(zip(inputs_token['input_ids'], inputs_token['attention_mask'], labels)), batch_size=30, collate_fn=collate_batch)

In [113]:
for input_id,attn_mask,label in loader:
  break

In [114]:
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,  #original transformer word token embedding
                n_tokens: int = 10, #number of tokens for each task
                random_range: float = 0.5, #range to init embedding
                initialize_from_vocab: bool = True):
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens, 
                                                                               random_range, 
                                                                               initialize_from_vocab))
        #self.learned_embedding: (n_tokens, emb dim)
        #self.wte.weight:        (vocab size, emb dim)
        
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
    
    #define the forward process
    def forward(self, tokens):
        #first get the embedding of the input text, which is the n_tokens:
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        #input_embedding: b, input_len, emb dim
        
        #repeat the learned embedding to all batch size
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        #input_embedding: b, n_tokens, emb dim
        
        concat_embed = torch.cat([learned_embedding, input_embedding], 1)
        #concat_embed: 1, input_len + n_tokens, emb dim
        
        return concat_embed

In [115]:
model_setup_for_prompt_tuning=False
if not model_setup_for_prompt_tuning:
  model.train()

  # Freeze model
  for param in model.parameters():
      param.requires_grad = False

  old_wte = model.get_input_embeddings()

  # Add softembedding module
  s_wte = SoftEmbedding(old_wte,
                        n_tokens=prefix_len,
                        initialize_from_vocab=True).to("cuda")
  model.set_input_embeddings(s_wte)

  # Set up optimizer
  params = [model.transformer.wte.learned_embedding]
  optimizer = Adafactor(params=params)
  

In [ ]:
inputs['input_id']
inputs['attention_mask']

In [122]:
iterations = 4000
input = {}
for input_id,attn_mask,label in loader:
  for i in range(iterations):
      optimizer.zero_grad()
      input['input_ids'] = input_id
      input['attention_mask'] = attn_mask
      output = model(**input, labels=label)
      loss = output.loss
      loss.backward()
      optimizer.step()
      if i%10 == 0:
        print(f"{i}: Loss: {loss}")

RuntimeError: ignored